In [ ]:
import os
import pickle
import collections
import datetime
import copy
import numpy as np
import scipy as sp
import pandas as pd
import happybase
import tensorflow as tf
import warnings
import re
import sklearn.model_selection
import sklearn.preprocessing

In [ ]:
data_dir = r"/mnt1/zhaodachuan/data/predict_user_attribute/data/raw_data"

In [ ]:
file_list = []
for root,_,path_list in os.walk(data_dir):
    for path in path_list:
        file_list.append(os.path.join(root,path))
print(file_list)

In [ ]:
%%time
#读取用户数据
all_file_data = []
for file_name in file_list:
    all_file_data.append(pd.read_csv(os.path.join(data_dir,file_name),sep='\001',header=None,error_bad_lines=False))
df = pd.concat(all_file_data,axis=0,ignore_index=True)
del all_file_data
column_list = ['用户id','用户姓名','用户性别','用户出生时间','所在公司','所处职位','行业','微信网名','微信性别','常驻省','常驻市','是否付费用户','付费等级','关注的作者','作者类型','关注的马甲','关注的公司','文章id','是否精选文章','浏览文章时间','浏览时长','文章频道','文章类型','文章标签','文章图片数量','文章字数','新定位省','新定位市','评论内容','打赏金额','设备型号','使用网络']
df.columns = column_list
old_df = copy.deepcopy(df)

In [ ]:
use_column_list = ["用户id",'行业',"常驻省","常驻市","所在公司","所处职位","关注的公司","新定位省","新定位市","浏览时长","文章标签","文章类型","评论内容"]

In [ ]:
df = copy.deepcopy(old_df)

In [ ]:
df = df[use_column_list]
df

# 数据分析

In [ ]:
df.dropna(subset=["所处职位"],inplace=True)
df

In [ ]:
df.fillna("",inplace=True)

In [ ]:
def temp_function(v):
    if len(str(v)) >= 20 or str(v) == "-1":
        return False
    else:
        return True

In [ ]:
df = df[df["所处职位"].apply(lambda x :temp_function(x))]

In [ ]:
df

In [ ]:
rule = re.compile("^[a-zA-z]{1}.*$")

def is_startswith_en(_str):
    if rule.match(_str) is not None:
        return True
    else:
        return False
def is_num(_str):
    if bool(re.search(r'\d', _str)):
        return True
    else:
        return False

In [ ]:
def temp_function(v:str):
    v = str(v)
    res = False
    if is_startswith_en(v):
        pass
    elif is_num(v):
        res = True
    else:
        if (not "小姐" in v) and (not "开票" in v) and (not "一夜情" in v) and (not "按摩" in v) and (not "会所" in v) and \
        (not "小妹" in v) and (not "q" in v) and (not "Q" in v) and (not "www" in v) and (not "出台" in v) and \
         (not "信用卡" in v) and (not "正妹" in v) and (not "com" in v) and (not "http" in v) and (len(v) < 15) and (not "微信" in v):
            res = True
    return res

In [ ]:
df = df[df["所处职位"].apply(lambda x :temp_function(x))]

In [ ]:
df

In [ ]:
# replace_dict = {}
# for v in boss_set:
#     replace_dict[v] = '企业创始人/CEO'
# for v in decison_set:
#     replace_dict[v] = "企业业务决策层"
# for v in common_decision_set:
#     replace_dict[v] = "企业一般管理人员"
# for v in shiye_staff_set:
#     replace_dict[v] = "企业普通员工"
# for v in staff_set:
#     replace_dict[v] = "事业单位员工"
# for v in student_set:
#     replace_dict[v] = "高校学生"
# for v in freedom_set:
#     replace_dict[v] = "自由职业"


In [ ]:
# train_df = df[df['所处职位'].apply(lambda x : x in xxx_set)]

In [ ]:
# df['所处职位'].replace(replace_dict,inplace=False)

In [ ]:
v_list = [str(v) for v in df["所处职位"].unique()]

In [ ]:
v_list

In [ ]:
# 企业创始人/CEO
original_set = set(v_list)
boss_list = ["创始人", "ceo", "CEO", "合伙人", "老板", "董事", "主席", "股东", "总裁", "总经理","副总经理"]
boss_set = set()
for i in list(original_set):
    for j in boss_list:
        if re.search(j, i):
            boss_set.add(i)
            original_set.remove(i)
            break

In [ ]:
# 企业业务决策层
decison_list = ["首席", "副总","经理","CTO", "CMO", "CFO", "cto", "cmo", "cfo"]
decison_set = set()
for i in list(original_set):
    for j in decison_list:
        if re.search(j, i):
            decison_set.add(i)
            original_set.remove(i)
            break


In [ ]:
# 企业一般管理人员
common_decision_list = ["总监", "主管", "负责人", "主编",]
common_decision_set = set()
for i in list(original_set):
    for j in common_decision_list:
        if re.search(j, i):
            common_decision_set.add(i)
            original_set.remove(i)
            break


In [ ]:
#  事业单位员工
shiye_staff_list = ["银行", "职员", "老师", "教授", "医生", "律师", "护士", "主任", "处长","研究院",'委',"局"]
shiye_staff_set = set()
for i in list(original_set):
    for j in shiye_staff_list:
        if re.search(j, i):
            shiye_staff_set.add(i)
            original_set.remove(i)
            break


In [ ]:
# 企业普通员工
staff_list = ["工程师", "员", "编辑", "运营", "助理", "销售", "司机", "顾问", "记者", "分析师", "秘书", "会计","师","策划","工","编", "实习生","研发","技术"
             "推广", "公关","市场","产品","测试","运维","物流","前端","后端", "研发","技术","指导","专家","分析","研究","广告","咨询"]
staff_set = set()
for i in list(original_set):
    for j in staff_list:
        if re.search(j, i):
            staff_set.add(i)
            original_set.remove(i)
            break


In [ ]:
# 高校学生
student_list = ["本科生", "大一", "大二", "大三", "大四", "研究生", "博士","学生"]
student_set = set()
for i in list(original_set):
    for j in student_list:
        if re.search(j, i):
            student_set.add(i)
            original_set.remove(i)
            break


In [ ]:
# 自由职业者
freedom_list = ["自", "自由"]
freedom_set = set()
for i in list(original_set):
    for j in freedom_list:
        if re.search(j, i):
            freedom_set.add(i)
            original_set.remove(i)
            break


In [ ]:
original_set
original_list01 = list(original_set)
original_list01

In [40]:
replace_dict = {}
for v in boss_set:
    replace_dict[v] = '企业创始人/CEO'
for v in decison_set:
    replace_dict[v] = "企业业务决策层"
for v in common_decision_set:
    replace_dict[v] = "企业一般管理人员"
for v in shiye_staff_set:
    replace_dict[v] = "企业普通员工"
for v in staff_set:
    replace_dict[v] = "事业单位员工"
for v in student_set:
    replace_dict[v] = "高校学生"
for v in freedom_set:
    replace_dict[v] = "自由职业"    

In [41]:
df['所处职位'].replace(replace_dict,inplace=True)

/home/zhangwanli/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [42]:
df

,用户id,行业,常驻省,常驻市,所在公司,所处职位,关注的公司,新定位省,新定位市,浏览时长,文章标签,文章类型,评论内容
10253,100078,-1,-1,-1,厦门房网,事业单位员工,-1,-1,-1,-1,-1,-1,-1
10739,10025,-1,-1,-1,片刻,企业业务决策层,-1,-1,-1,-1,-1,-1,-1
10936,10032,-1,-1,-1,巨象网,事业单位员工,-1,-1,-1,-1,-1,-1,-1
11509,100526,-1,-1,-1,676787,39209934,-1,-1,-1,-1,-1,-1,-1
11857,10065,-1,-1,-1,govmade,小二,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14634509,98,-1,-1,-1,汉森供应链、农特集团,企业创始人/CEO,-1,-1,-1,-1,-1,-1,-1
14634976,98149,-1,-1,-1,拜耳材料科技,企业业务决策层,-1,-1,-1,-1,-1,-1,-1
14642274,99083,-1,-1,-1,鸿富锦精密工业,事业单位员工,-1,-1,-1,-1,-1,-1,-1
14643119,99245,-1,-1,-1,895457470,9000043,-1,-1,-1,-1,-1,-1,-1


In [43]:
career_list = ['企业创始人/CEO', "企业业务决策层", "企业一般管理人员", "企业普通员工","事业单位员工","高校学生","自由职业" ]

In [44]:
train_df = df[df['所处职位'].apply(lambda x : x in career_list)]
train_df

,用户id,行业,常驻省,常驻市,所在公司,所处职位,关注的公司,新定位省,新定位市,浏览时长,文章标签,文章类型,评论内容
10253,100078,-1,-1,-1,厦门房网,事业单位员工,-1,-1,-1,-1,-1,-1,-1
10739,10025,-1,-1,-1,片刻,企业业务决策层,-1,-1,-1,-1,-1,-1,-1
10936,10032,-1,-1,-1,巨象网,事业单位员工,-1,-1,-1,-1,-1,-1,-1
12227,100782,-1,-1,-1,阿里巴巴,事业单位员工,-1,-1,-1,-1,-1,-1,-1
13581,101259,-1,-1,-1,宜信惠琮国际融资租赁有限公司,企业业务决策层,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14627692,97089,-1,-1,-1,广东顺德时代易家电子商务有限公司,企业一般管理人员,-1,-1,-1,-1,-1,-1,-1
14631235,97977,-1,-1,-1,兰博基尼,企业业务决策层,-1,-1,-1,-1,-1,-1,-1
14634509,98,-1,-1,-1,汉森供应链、农特集团,企业创始人/CEO,-1,-1,-1,-1,-1,-1,-1
14634976,98149,-1,-1,-1,拜耳材料科技,企业业务决策层,-1,-1,-1,-1,-1,-1,-1


In [ ]:
list1 = train_df["所处职位"].values.tolist()
dict(zip(*np.unique(list1, return_counts=True)))

In [ ]:
train_df["行业"].unique()

In [ ]:
train_df["常驻省"].unique()

In [ ]:
train_df["浏览时长"].unique()

In [ ]:
train_df["常驻市"].unique()

In [ ]:
train_df["关注的公司"].unique()

In [ ]:
train_df["新定位省"].unique()

In [ ]:
train_df["新定位市"].unique()

In [ ]:
train_df["文章标签"].unique()

In [ ]:
train_df["文章类型"].unique()

In [ ]:
train_df = train_df[["用户id","所处职位", "关注的公司","行业"]]

train_df

In [ ]:
train_df.fillna("",inplace=True)

In [ ]:
train_df["关注的公司"] = train_df["关注的公司"].apply(lambda x : str(x))
train_df["行业"] = train_df["行业"].apply(lambda x : str(x))
train_df

In [ ]:
train_df["关注的公司"].unique()

In [ ]:
train_df["行业"].unique()

In [33]:
use_column_list =["用户id","所处职位","关注的公司","行业"]
y_col = "所处职位"

In [ ]:
#训练数据
multi_hot_column_odict = collections.OrderedDict()

multi_hot_column_set = set([v for v in use_column_list if v != y_col and v!="用户id"])
for i,column in enumerate(multi_hot_column_set):
    multi_hot_column_odict[column] = sklearn.preprocessing.MultiLabelBinarizer()
    split_column_list = [item.split(",") if isinstance(item,str) else str(item) for item in train_df[column]]
    #multi_hot_column_odict[column].fit(split_column_list)
    train_df[column] = multi_hot_column_odict[column].fit_transform(split_column_list).tolist()

In [ ]:
%%time
y_onehot = sklearn.preprocessing.LabelBinarizer()
y_onehot.fit(train_df[y_col])


In [ ]:
y_onehot.transform(train_df[y_col]).tolist()

In [ ]:
y_onehot.transform(['事业单位员工'])

In [ ]:
y_onehot.inverse_transform(np.array([[1, 0, 0, 0, 0, 0, 0]]))

In [ ]:
y_onehot.inverse_transform(np.array([[0.6, 0.4, 0, 0, 0, 0, 0]])).tolist()[0]

In [ ]:
train_df[y_col] = y_onehot.transform(train_df[y_col]).tolist()

In [ ]:
train_df

In [ ]:
for k,v in multi_hot_column_odict.items():
    print(k,len(v.classes_))

In [ ]:
train, test = sklearn.model_selection.train_test_split(train_df, test_size=0.2)
train, val = sklearn.model_selection.train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
def generate_xy(data_df, y_col, multi_hot_column_odict, shuffle=True):
    """"""
    x_df = data_df.copy()
    x_list = []
    y_list = []
    for i,index in enumerate(x_df.index):
        temp_x_list = np.concatenate(x_df.loc[index,[column for column in multi_hot_column_odict]]).tolist()
        temp_y_list = x_df.loc[index,y_col]
        x_list.append(temp_x_list)
        y_list.append(temp_y_list)
        if i % 10000 == 0:
            print(i)
    return np.array(x_list,dtype=np.float32), np.array(y_list,dtype=np.int32)

In [ ]:
%%time
x_train,y_train = generate_xy(train,y_col,multi_hot_column_odict)
x_vali,y_valid = generate_xy(val,y_col,multi_hot_column_odict)
x_test,y_test = generate_xy(test,y_col,multi_hot_column_odict)

In [ ]:
input_dim = 0
for k,v in  multi_hot_column_odict.items():
    input_dim += len(v.classes_)
input_dim

In [ ]:
y_dim = len(y_onehot.classes_)
y_dim

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(input_dim,)))
model.add(tf.keras.layers.Dense(y_dim, activation='softmax',use_bias=True,
                                #kernel_regularizer=tf.keras.regularizers.l1(0.01)
                               ))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
model.summary()

In [ ]:
x_train

In [ ]:
collections.Counter([str(v) for v in y_train])

In [ ]:
model.fit(x_train, 
          y_train, 
          batch_size=64,
          epochs=20,
         validation_data = (x_vali,y_valid))

In [ ]:
#保存模型
with open("multi_hot_column_odict.pkl","wb") as f:
    pickle.dump(multi_hot_column_odict,f)
with open("y_onehot.pkl","wb") as f:
    pickle.dump(y_onehot,f)
model.save("model_output")

In [52]:
train_df

,用户id,行业,常驻省,常驻市,所在公司,所处职位,关注的公司,新定位省,新定位市,浏览时长,文章标签,文章类型,评论内容
10253,100078,-1,-1,-1,厦门房网,"[1, 0, 0, 0, 0, 0, 0]",-1,-1,-1,-1,-1,-1,-1
10739,10025,-1,-1,-1,片刻,"[0, 0, 1, 0, 0, 0, 0]",-1,-1,-1,-1,-1,-1,-1
10936,10032,-1,-1,-1,巨象网,"[1, 0, 0, 0, 0, 0, 0]",-1,-1,-1,-1,-1,-1,-1
12227,100782,-1,-1,-1,阿里巴巴,"[1, 0, 0, 0, 0, 0, 0]",-1,-1,-1,-1,-1,-1,-1
13581,101259,-1,-1,-1,宜信惠琮国际融资租赁有限公司,"[0, 0, 1, 0, 0, 0, 0]",-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14627692,97089,-1,-1,-1,广东顺德时代易家电子商务有限公司,"[0, 1, 0, 0, 0, 0, 0]",-1,-1,-1,-1,-1,-1,-1
14631235,97977,-1,-1,-1,兰博基尼,"[0, 0, 1, 0, 0, 0, 0]",-1,-1,-1,-1,-1,-1,-1
14634509,98,-1,-1,-1,汉森供应链、农特集团,"[0, 0, 0, 1, 0, 0, 0]",-1,-1,-1,-1,-1,-1,-1
14634976,98149,-1,-1,-1,拜耳材料科技,"[0, 0, 1, 0, 0, 0, 0]",-1,-1,-1,-1,-1,-1,-1


In [64]:
import happybase
pool = happybase.ConnectionPool(size=100, host="47.110.125.69", port=9090, timeout=None,
                                protocol='binary'
                                )
with pool.connection() as conn:
    table = conn.table("feature_center:user_profile")
    with table.batch(batch_size=400) as batch:
        for i,index in enumerate(train_df.index):
            batch.put(str(train_df.loc[index,"用户id"]).encode(), {"attribute:input_position".encode(): y_onehot.inverse_transform(np.array([train_df.loc[index,y_col]]))[0].encode()})
            #print(str(train_df.loc[index,"用户id"]).encode(),{"attribute:input_position".encode(): y_onehot.inverse_transform(np.array([train_df.loc[index,y_col]]))[0].encode()})
            #break
            #batch.delete(str(train_df.loc[index,y_col]).encode())
            